In [39]:
import requests
from bs4 import BeautifulSoup

username = "mariagps"
user_password = "8Z4vn*!WTP"
role = "editor"
login_data = {"username": username, "password": user_password,  "role": "editor",
              "hdnManuscriptServicesDisplayed" : "false",   'hdnNeedMultipLoginDropdown' : 0,
              'hdnOrcidIsAuthenticated' : 'False', 'hdnOrcidIsDuplicateEmail' : 'False',
              'hdnSsoLoginEnabled' : 'False', 'hdnUseOrcideLogin' : 'True', 'ignoreWarning' : 0,
              'initiateAscoLogin' : 'False', 'orcidAuthenticated' : 0, 'orcidLogin' : 0
 }
#login_data = {"username": username, "password": user_password, "Login": "submit", "role": "editor"}
headers = {
'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
'Accept-Encoding': 'gzip, deflate, br',
'Accept': '*/*',
'Connection': 'keep-alive',
'sec-ch-ua' : '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
'sec-ch-ua-mobile' : '?0',
'sec-ch-ua-platform' : '"Windows"',
'DNT' : '1',
'Upgrade-Insecure-Requests' : '1',
'Content-Type' : 'application/x-www-form-urlencoded',
'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36',

}

## login 

url = "https://www.editorialmanager.com/nefro/LoginAction.ashx" #este funciona

session = requests.Session()
response = session.get(url)
print(response.status_code)
#soup = BeautifulSoup(response.content, features="html.parser")
#print(soup)
states = ["__RequestVerificationToken", "Email", "RememberMe"]


post_request = session.post(url,  data=login_data, headers=headers, allow_redirects=True)

#print cookies
cookies = session.cookies.get_dict()
print(cookies)


200
{'EMSessionID': '%7B899EE9BE%2DDA4F%2D4C65%2D99B5%2DFB7529825EC3%7D'}


In [40]:
#save html
#with open("login.html", "w") as f:
    #f.write(post_request.text)


No veo el html de la página, parece sólo js

In [41]:
'''#call the main page and save html
url = "https://www.editorialmanager.com/nefro/default2.aspx"
response = session.get(url)
with open("main.html", "w") as f:
    f.write(response.text)'''

'#call the main page and save html\nurl = "https://www.editorialmanager.com/nefro/default2.aspx"\nresponse = session.get(url)\nwith open("main.html", "w") as f:\n    f.write(response.text)'

Aquí parece que entro en la página, no me muestra la cabecera pero no hay un link de login, así que asumo que estoy logueado. Pruebo a meterme en Menú ppal

In [42]:
#call manin menu and save html
url = 'https://www.editorialmanager.com/nefro/manuscript_status.asp'
response = session.get(url)
'''#with open("menu.html", "w") as f:
    f.write(response.text)'''
response

<Response [200]>

Efectivamente, entro en el Menú ppal de autor

Ya sé hacer login en la web pero en la página de autor. Tengo que cambiar a editor.

In [43]:
#url = "https://www.editorialmanager.com/nefro/Default.aspx?pg=EditorsMainMenu.aspx"
#url = "https://www.editorialmanager.com/nefro/default2.aspx"
url = "https://www.editorialmanager.com/nefro/manuscript_status.asp"
data = {'role': 'editor',}
 #   '__EVENTTARGET': 'RoleDropdown',
#add defaultMenu to cookies: defaultMenu=0; Path=/nefro/; Domain=editorialmanager.com; Secure;
cookies['defaultMenu'] = '0'
cookies['CurrentLang'] = 'es-ES'
cookies['Path'] = '/nefro/'
cookies['Domain'] = 'editorialmanager.com'


headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    #add the cookie from the variable cookies
    'Cookie': 'CurrentLang=es-ES; EMSessionID='+cookies['EMSessionID'],
    #'Cookie': 'CurrentLang=es-ES; EMSessionID=%7B72CDCD21%2D8534%2D4800%2D9B32%2D719C1E125623%7D'
}

response = session.post(url, data=data, headers=headers, cookies=cookies, allow_redirects=True)
print(response.status_code)

'''with open("Role_editor.html", "w") as f:
    f.write(response.text)'''



200


'with open("Role_editor.html", "w") as f:\n    f.write(response.text)'

In [68]:
#manuscritos asignados
url = 'https://www.editorialmanager.com/nefro/AllAssignedSubmissions.aspx'

response = session.post(url, data=data, headers=headers, cookies=cookies, allow_redirects=True)
print(response.status_code)

body = response.text

import re

pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>.*?</td>')
matches = pattern.findall(body)

#with open("manuscritos_asignados.html", "w") as f:
    #f.write(response.text)
# if matches is not empty print ok
if matches:
    print("Descargado correctamente")


200
Descargado correctamente


No lo veo en el navegador, pero parece que sí que me he traído el html de manuscritos asignados. Intento scrapearlo.

Lo que necesito está dentro de un iframe #document ==$0

In [69]:
#create a dataframe called df_Asignados with the columns: numero_manuscrito, tipo_articulo, titulo, autor, fecha_inicial, fecha_estado, estado_actual, editor, estado_evaluacion, decision_editor
import pandas as pd
df_asignados = pd.DataFrame(columns=['numero_manuscrito', 'tipo_articulo', 'titulo', 'autor', 'fecha_inicial', 
                'fecha_estado', 'estado_actual', 'editor', 'estado_evaluacion', 'decision_editor','doc_id', 'Completo', 'Aceptado', 'Invitado – Sin Respuesta', 'Rechazado', 'Tarde', 'data_identity'])

#the columns 10-15 are integers
#df_asignados[['doc_id', 'Completo', 'Aceptado', 'Invitado – Sin Respuesta', 'Rechazado', 'Tarde']] = df_asignados[['doc_id', 'Completo', 'Aceptado', 'Invitado – Sin Respuesta', 'Rechazado', 'Tarde']].astype(int)

In [70]:
#function that find all the rs-headerRow elements
def find_estado_evaluacion(estado_ev_test, id):
    estado_evaluacion_elemento = []
    soup = BeautifulSoup(estado_ev_test, 'html.parser')
    #print('soup',soup.prettify())
    
    #rs_headerRow = soup.find_all('div', class_="rs-headerRow")
    #print('rs_headerRow', rs_headerRow)

    #*****************************************************
    '''    #delete the docid part of the code becaus I will use the data-identity

    #find the dta-docid from <div class='reviewerHoverDetails leftAligned' data-docid='15961'>
    docid = soup.find('div', class_="reviewerHoverDetails leftAligned")['data-docid']
    #print('doc id', docid)
    #append the docid to the list estado_evaluacion_elemento
    estado_evaluacion_elemento.append('docid')
    estado_evaluacion_elemento.append(docid)

    #add th id to the dataframe
    df_asignados.loc[id, 'doc_id'] = docid'''
    #*****************************************************

    rows = soup.select('.reviewerHoverDetails .rs-headerRow')
    for row in rows:
        count = row.select_one('.rs-countCell span').text
        detail = row.select_one('.rs-detailCell span').text
        #print(count, detail)
        #append the detail and count to the list estado_evaluacion_elemento
        estado_evaluacion_elemento.append(detail)
        estado_evaluacion_elemento.append(count)
        # add the count to the dataframe in the column detail
        df_asignados.loc[id, detail] = count


    return



In [74]:
#extract the values from the matches
for i in range(len(matches)):
    pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;;\'>')
    values = pattern.split(matches[i])
    #remove the first element because it is empty
    values.pop(0)
    #print(values[1])
    for j in range (len(values)):
        #print(j, values[j])
        #Titulo and autor are in the same field. If the value is the titulo, split in two and save in the corresponding column
        if j == 2:
            #split in two the string by <td class='  ' style='vertical-align: middle;text-align: center;'> 
            pattern = re.compile(r'<td class=\'  \' style=\'vertical-align: middle;text-align: center;\'>')
            title_author = pattern.split(values[j])
            #print(title_author)
            #store in the corresponding column
            df_asignados.loc[i, 'titulo'] = title_author[0]
            df_asignados.loc[i, 'autor'] = title_author[1]

        else:
            #if the column is >2, add 1 to the column number
            if j > 2:
                df_asignados.loc[i, df_asignados.columns[j+1]] = values[j]
            else:

                df_asignados.loc[i, df_asignados.columns[j]] = values[j]
    
    #pattern = re.compile(r'<tr class=\'\' id=\'fr[0-9]+\' data-identity=\'[0-9]+\' data-rowindex=\'[0-9]+\'')
    #matches = pattern.findall(body)
    #extract the data-identity value from data-identity='****'. Keep just the numbers between the single quotes
    #data_identity = re.findall(r'data-identity=\'([0-9]+)\'', matches[i])[0]
    #print('data_identity, ', data_identity)
    #store the data-identity in the dataframe
    #df_asignados.loc[i, 'data_identity'] = data_identity
    

    #expand estado_evaluacion to columns: doc_id, Completo, Aceptado, Invitado, Rechazado, Tarde
    estado_evaluacion = df_asignados.loc[i, 'estado_evaluacion']
    #print(estado_evaluacion)
    if estado_evaluacion != '':
        find_estado_evaluacion(estado_evaluacion, i)
        
    #extract decision_editor
    decision_editor = df_asignados.loc[i, 'decision_editor']
    #print(decision_editor)
    try:
        #extract link text from the decision_editor_test
        soup_link = BeautifulSoup(decision_editor, 'html.parser')
        link = soup_link.find('a').text
        #print(link)
        #add the link to the dataframe
        df_asignados.loc[i, 'decision_editor'] = link
    except:
        df_asignados.loc[i, 'decision_editor'] = ""

    #in body, we have the data-identity value inside:
    # The lines are in id='fr***' data-identity='****' data-rowindex
    #we need to extract the data-identity value ****
    
    #find the lines with the data-identity value
    pattern = re.compile(r'id=\'fr[0-9]+\' data-identity=\'[0-9]+\' data-rowindex=\'[0-9]+\'')
    matches = pattern.findall(body)

    #find the values 
    #for i in range(len(matches)):

    #extract the data-identity value from data-identity='****'. Keep just the numbers between the single quotes
    data_identity = re.findall(r'data-identity=\'([0-9]+)\'', matches[i])[0]
    #print( data_identity)
    #store the data-identity in the dataframe
    df_asignados.loc[i, 'data_identity'] = data_identity

    
    

#df_asignados.head()


In [75]:
#remove the column estado_evaluacion
df_asignados.drop('estado_evaluacion', axis=1, inplace=True)
#change the names of the columns Completo, Aceptado, Invitado – Sin Respuesta, Rechazado, Tarde to est_ev_completo, est_ev_aceptado, est_ev_invitado, est_ev_rechazado, est_ev_tarde
df_asignados.rename(columns={'Completo': 'est_ev_completo', 'Aceptado': 'est_ev_aceptado', 'Invitado – Sin Respuesta': 'est_ev_invitado', 'Rechazado': 'est_ev_rechazado', 'Tarde': 'est_ev_tarde'}, inplace=True)
#convert NaN to 0
df_asignados.fillna(0, inplace=True)

In [76]:
df_asignados

,numero_manuscrito,tipo_articulo,titulo,autor,fecha_inicial,fecha_estado,estado_actual,editor,decision_editor,doc_id,est_ev_completo,est_ev_aceptado,est_ev_invitado,est_ev_rechazado,est_ev_tarde,data_identity
0,NEFRO-D-23-00010,Original,The Association of Vitamin D Supplementation i...,"Ilhan Kilic, M.D.",09/01/2023,10/01/2023,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16081
1,NEFRO-D-23-00009,Notas técnicas,Actualización en el tratamiento con fluidos de...,"Alejandro Gonzalez-Castro, Ph.D, M.D.",09/01/2023,10/01/2023,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16082
2,NEFRO-D-23-00008,Original,Correlation of Luminex-based single-antigen-ba...,"Suleyman Rustu Rustu Oguz, MD PhD",06/01/2023,09/01/2023,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16079
3,NEFRO-D-23-00006,Original,Therapeutic adherence in kidney transplanted a...,Ana Barbosa Rodrigues,04/01/2023,05/01/2023,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16078
4,NEFRO-D-23-00005,Revisiones cortas,ANEMIA RENAL: TRATAMIENTOS ACTUALES Y MOLÉCULA...,"MANUEL HERAS BENITO, M.D",04/01/2023,05/01/2023,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16077
5,NEFRO-D-23-00002,Original,Sobrevida y su relación con el tipo de transfe...,"Rafael Alberto Gómez Acevedo, M.D.",31/12/2022,10/01/2023,Pendiente del Editor,Miguel Perez Fontan,,0,0,0,0,0,0,16068
6,NEFRO-D-22-00457,Cartas al editor,Dying with end stage kidney disease: factors a...,"Ana Cunha Rodrigues, M.D.",30/12/2022,31/12/2022,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16070
7,NEFRO-D-22-00455,Original,El desarrollo de una lesión renal aguda es un ...,"Borja Quiroga, MD PhD",28/12/2022,28/12/2022,Pendiente del Editor,Mariano Rodriguez,,0,0,0,0,0,0,16061
8,NEFRO-D-22-00454,Original,Activación y control de enfermedad de paciente...,Alicia Cana,23/12/2022,01/01/2023,Pendiente Evaluadores,Mariano Rodriguez,,0,1,1,1,1,0,16058
9,NEFRO-D-22-00448,Original breve,EXPRESIÓN DE C4d EN GLOMERULOESCLEROSIS FOCAL ...,"Victoriano Pérez-Vázquez, Ph.D",20/12/2022,31/12/2022,Pendiente del Editor,Marian Goicoechea,,0,0,0,0,0,0,16048


In [77]:
#export to excel
df_asignados.to_excel('asignados.xlsx', index=False)

# Bajar información de manuscritos asignados individualmente

In [ ]:
# https://www.editorialmanager.com/nefro/doc_history.asp?docid=16089&ms_num=%27NEFRO-D-23-00014%27
# the url is the same for all the documents. The only thing that changes is the data-identity and numero_manuscrito



In [14]:
#scrap the data from the url  https://www.editorialmanager.com/nefro/doc_history.asp?docid=16089&ms_num=%27NEFRO-D-23-00014%27

url = 'https://www.editorialmanager.com/nefro/doc_history.asp?docid=16089&ms_num=%27NEFRO-D-23-00014%27'

#open the url
session = requests.Session()
response = session.get(url, headers=headers)
print(response.status_code)

#extract the body
body = response.text


200


In [24]:
#There are two tables in the class= 'datatable' in the body
#scrap the second table

data = []

numero_manuscrito = 

soup = BeautifulSoup(body, 'html.parser')
table = soup.find_all('table', class_='datatable')[1]
#print(table)

rows = table.find_all("tr")

for row in rows:
    cells = row.find_all("td")
    if len(cells)>0:
        fecha = cells[0].text.strip()
        carta = cells[1].text.strip()
        destinatario = cells[2].text.strip()
        estado = cells[3].text.strip()
        revision = cells[4].text.strip()
        operador = cells[5].text.strip()
        
        data.append({"Fecha": fecha, "Carta": carta, "Destinatario": destinatario, "Estado": estado, "Revision": revision, "Operador": operador})

df = pd.DataFrame.from_dict(data)
df


,Fecha,Carta,Destinatario,Estado,Revision,Operador
0,12/01/2023,Para director: nuevo artículo,Mariano Rodriguez,Pendiente del Editor,0,Sandra Gallego
1,12/01/2023,Para autor: acuse de recibo (con nº ref.),"Mehmet Ali Narsat, MD",Nuevo envio / New submission,0,Sandra Gallego
2,12/01/2023,Author - Author other author verification,Eren YILDIZ,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
3,12/01/2023,Author - Author other author verification,sedat GÜLTEN,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
4,12/01/2023,Para asist. ed. o editor: entrada nuevo manusc...,Amritha Ganga M.G,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
5,12/01/2023,Para asist. ed. o editor: entrada nuevo manusc...,Ragul Mani,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
6,12/01/2023,Para asist. ed. o editor: entrada nuevo manusc...,Balamurugan M,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
7,12/01/2023,Para asist. ed. o editor: entrada nuevo manusc...,Yogambal Mangali,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
8,12/01/2023,Para asist. ed. o editor: entrada nuevo manusc...,Abarna Murugan,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
9,12/01/2023,Para asist. ed. o editor: entrada nuevo manusc...,Surjith Kumar Kumar,Nuevo envio / New submission,0,"Mehmet Ali Narsat, MD"
